<a href="https://colab.research.google.com/github/YianKim/2022_uncertainty_aware_semisupervise/blob/main/Keras_UPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras import optimizers
from keras.callbacks import *
from sklearn.metrics import *
from keras.models import load_model

import tensorflow as tf
import tensorflow.keras.backend as backend
import math
import gc

from tqdm import tqdm

# cifar10 dataset

In [ ]:
cifar10 = keras.datasets.cifar10 
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images/255
test_images = test_images/255

In [ ]:
def dummy_labels(labels):
  zero_labels = np.zeros([labels.shape[0], 10], np.int8)  
  for i in range(labels.shape[0]):
    zero_labels[i][labels[i]] = 1
  return(zero_labels)

In [ ]:
train_labels = dummy_labels(train_labels)
test_labels = dummy_labels(test_labels)

In [ ]:
# 1000 labeled, 49000 unlabeled
indx = random.sample(range(train_labels.shape[0]),train_labels.shape[0])

lbl_train_images = train_images[indx[:1000]]
ubl_train_images = train_images[indx[1000:]]

lbl_train_labels = train_labels[indx[:1000]]
ubl_train_labels = train_labels[indx[1000:]]

# valids1 =  train_images[indx[800:1000]]
# valids2 =  train_labels[indx[800:1000]]

# pseudo labeling

### 스케줄러

In [ ]:
class SGDR(Callback):
    """This callback implements the learning rate schedule for
    Stochastic Gradient Descent with warm Restarts (SGDR),
    as proposed by Loshchilov & Hutter (https://arxiv.org/abs/1608.03983).
    
    The learning rate at each epoch is computed as:
    lr(i) = min_lr + 0.5 * (max_lr - min_lr) * (1 + cos(pi * i/num_epochs))
    
    Here, num_epochs is the number of epochs in the current cycle, which starts
    with base_epochs initially and is multiplied by mul_epochs after each cycle.
    
    # Example
        ```python
            sgdr = CyclicLR(min_lr=0.0, max_lr=0.05,
                                base_epochs=10, mul_epochs=2)
            model.compile(optimizer=keras.optimizers.SGD(decay=1e-4, momentum=0.9),
                          loss=loss)
            model.fit(X_train, Y_train, callbacks=[sgdr])
        ```
    
    # Arguments
        min_lr: minimum learning rate reached at the end of each cycle.
        max_lr: maximum learning rate used at the beginning of each cycle.
        base_epochs: number of epochs in the first cycle.
        mul_epochs: factor with which the number of epochs is multiplied
                after each cycle.
    """

    def __init__(self, min_lr=0.0, max_lr=0.03, base_epochs=20, mul_epochs=2):
        super(SGDR, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.base_epochs = base_epochs
        self.mul_epochs = mul_epochs

        self.cycles = 0.
        self.cycle_iterations = 0.
        self.trn_iterations = 0.

        self._reset()

    def _reset(self, new_min_lr=None, new_max_lr=None,
               new_base_epochs=None, new_mul_epochs=None):
        """Resets cycle iterations."""
        
        if new_min_lr != None:
            self.min_lr = new_min_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_base_epochs != None:
            self.base_epochs = new_base_epochs
        if new_mul_epochs != None:
            self.mul_epochs = new_mul_epochs
        self.cycles = 0.
        self.cycle_iterations = 0.
        
    def sgdr(self):
        
        cycle_epochs = self.base_epochs * (self.mul_epochs ** self.cycles)
        tide = ((self.cycles == 0) * 1) * (self.cycle_iterations*self.max_lr + (self.base_epochs - self.cycle_iterations)*self.min_lr) / self.base_epochs + ((self.cycles != 0) * 1)*(self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(np.pi * (self.cycle_iterations + 1) / cycle_epochs)))
        return tide
        
    def on_train_begin(self, logs=None):
        
        if self.cycle_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())
            
    def on_epoch_end(self, epoch, logs=None):
        
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        
        self.trn_iterations += 1
        self.cycle_iterations += 1
        if self.cycle_iterations >= self.base_epochs * (self.mul_epochs ** self.cycles):
            self.cycles += 1
            self.cycle_iterations = 0
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())

### main

In [ ]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def create_cnn_13():
  conv1a = Conv2D(128, (3,3), padding = 'same')
  bn1a = BatchNormalization()
  conv1b = Conv2D(128, (3,3), padding = 'same')
  bn1b = BatchNormalization()
  conv1c = Conv2D(128, (3,3), padding = 'same')
  bn1c = BatchNormalization()
  pl1 = MaxPooling2D(2, 2)
  MCdrop1 = Dropout(0.3)

  conv2a = Conv2D(256, (3,3), padding = 'same')
  bn2a = BatchNormalization()
  conv2b = Conv2D(256, (3,3), padding = 'same')
  bn2b = BatchNormalization()
  conv2c = Conv2D(256, (3,3), padding = 'same')
  bn2c = BatchNormalization()
  pl2 = MaxPooling2D(2, 2)
  MCdrop2 = Dropout(0.3)

  conv3a = Conv2D(512, (3,3))
  bn3a = BatchNormalization()
  conv3b = Conv2D(256, (1,1))
  bn3b = BatchNormalization()
  conv3c = Conv2D(128, (1,1))
  bn3c = BatchNormalization()
  pl3 = AveragePooling2D(6, 2)

  fc = Dense(10, activation='softmax')
  activ = keras.layers.ReLU()

  model = Sequential([
                      keras.Input(shape=(32, 32, 3)), 
                      conv1a, bn1a, activ,
                      conv1b, bn1b, activ,
                      conv1c, bn1c, activ,
                      pl1, MCdrop1,

                      conv2a, bn2a, activ,
                      conv2b, bn2b, activ,
                      conv2c, bn2c, activ,
                      pl2, MCdrop2,

                      conv3a, bn3a, activ,
                      conv3b, bn3b, activ,
                      conv3c, bn3c, activ,
                      pl3, Flatten(),
                      
                      fc
                      ])
  
  return model

def compile_cnn_13(model):

  opt = keras.optimizers.SGD(0.03, momentum=0.9)

  model.compile(
    optimizer = opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
  )

  return model

def cnn_13():

  model = create_cnn_13()
  model = compile_cnn_13(model)

  return model

def fit_and_labeling_cnn_13(Epoch, Batch):

  lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
  early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, mode='auto')
  sgdr = SGDR(min_lr=0.0, max_lr=0.03, base_epochs=20) #스케줄러
  
  model.fit(
      x=lbl_train_images,
      y=lbl_train_labels,
      epochs=Epoch,
      verbose=1,
#       validation_data = (valids1, valids2),
      batch_size=Batch,
#       callbacks=[sgdr, early_stopper]
      callbacks=[sgdr]
  )

  pred = model.predict(test_images)
  acc = (np.argmax(pred,axis=1) == np.argmax(test_labels,axis=1))*1
  print("test set 성능 : " + str(sum(acc)/len(acc)))
    
  for predsamples in tqdm(range(10)):
    if predsamples == 0 :
      predictions = model.predict(ubl_train_images)
      predictions = predictions.reshape((1,) + predictions.shape)
    else:
      pred = model.predict(ubl_train_images)
      pred = pred.reshape((1,) + pred.shape)
      predictions = np.concatenate((predictions, pred))

  return predictions

In [ ]:
def label_selecting():
  K_conf = 0.8
  K_uncert = 0.05

  pseudo = np.argmax(np.mean(predictions, axis=0), axis=1)
  conf = np.max(np.mean(predictions, axis=0), axis=1)
  uncert = np.std(predictions, axis=0)
  uncert = np.array([uncert[i][pseudo[i]] for i in range(len(pseudo))])

  select_pseudo = (1*(conf > K_conf)) * (1*(uncert < K_uncert))

  labels = []
  for i in pseudo:
    temp = [0,0,0,0,0,0,0,0,0,0]
    temp[i] = 1
    labels.append(temp)
  pseudo = np.array(labels)

  lbl_idx = []
  ubl_idx = []
  k = 0
  for i in select_pseudo:
    if i == 1:
      lbl_idx.append(k)
    if i == 0:
      ubl_idx.append(k)
    k += 1

    
  ubl_append = ubl_train_images[lbl_idx]
  pseudo_append = pseudo[lbl_idx]
    
  if itr < 10:
      numsamples = np.min(list(Counter(np.argmax(pseudo_append, axis=1)).values()))
      multlabel = np.argmax(pseudo_append, axis=1)
      sufindx = random.sample(range(len(multlabel)), len(multlabel))

      idxcounter = [0,0,0,0,0,0,0,0,0,0]
      idxsample = []

      for i in sufindx:
        if idxcounter[multlabel[i]] < numsamples+25:
          idxcounter[multlabel[i]] += 1
          idxsample.append(i)
      
      image1 = np.concatenate((lbl_train_images, ubl_append[idxsample]))
      label1 = np.concatenate((lbl_train_labels, pseudo_append[idxsample]))
  
  else:
      image1 = np.concatenate((lbl_train_images, ubl_append))
      label1 = np.concatenate((lbl_train_labels, pseudo_append))
  
  image2 = ubl_train_images[ubl_idx]

  return image1, label1, image2

In [ ]:
for itr in range(20):
  if itr == 0 :
    model = cnn_13()
  print(Counter(np.argmax(lbl_train_labels, axis=1)))
  predictions = fit_and_labeling_cnn_13(200, 128)
  lbl_train_images, lbl_train_labels, ubl_train_images = label_selecting()
  del predictions
  gc.collect()

Counter({0: 116, 8: 112, 7: 110, 1: 108, 9: 99, 5: 96, 2: 92, 3: 91, 4: 89, 6: 87})
Epoch 1/200
8/8 [==============================] - 5s 111ms/step - loss: 2.1845 - accuracy: 0.1820
Epoch 2/200
8/8 [==============================] - 0s 36ms/step - loss: 1.9781 - accuracy: 0.2900
Epoch 3/200
8/8 [==============================] - 0s 35ms/step - loss: 1.9959 - accuracy: 0.2510
Epoch 4/200
8/8 [==============================] - 0s 36ms/step - loss: 1.8584 - accuracy: 0.3050
Epoch 5/200
8/8 [==============================] - 0s 37ms/step - loss: 1.7394 - accuracy: 0.3600
Epoch 6/200
8/8 [==============================] - 0s 36ms/step - loss: 1.6465 - accuracy: 0.4250
Epoch 7/200
8/8 [==============================] - 0s 36ms/step - loss: 1.5806 - accuracy: 0.4290 0s - loss: 1.5763 - accuracy: 0.
Epoch 8/200
8/8 [==============================] - 0s 37ms/step - loss: 1.5264 - accuracy: 0.4700
Epoch 9/200
8/8 [==============================] - 0s 36ms/step - loss: 1.4553 - accuracy: 0.4790


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:28<00:00,  8.87s/it]


Counter({0: 2919, 8: 2915, 7: 2913, 1: 2911, 9: 2902, 5: 2899, 2: 2895, 4: 2892, 6: 2890, 3: 2869})
Epoch 1/200
227/227 [==============================] - 9s 40ms/step - loss: 0.5647 - accuracy: 0.8002
Epoch 2/200
227/227 [==============================] - 8s 37ms/step - loss: 0.3029 - accuracy: 0.8980
Epoch 3/200
227/227 [==============================] - 8s 37ms/step - loss: 0.1946 - accuracy: 0.9387
Epoch 4/200
227/227 [==============================] - 9s 38ms/step - loss: 0.1686 - accuracy: 0.9475
Epoch 5/200
227/227 [==============================] - 9s 38ms/step - loss: 0.1552 - accuracy: 0.9507
Epoch 6/200
227/227 [==============================] - 9s 37ms/step - loss: 0.1487 - accuracy: 0.9499
Epoch 7/200
227/227 [==============================] - 8s 37ms/step - loss: 0.1429 - accuracy: 0.9514
Epoch 8/200
227/227 [==============================] - 8s 37ms/step - loss: 0.1543 - accuracy: 0.9442
Epoch 9/200
227/227 [==============================] - 8s 36ms/step - loss: 0.1502 -

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:28<00:00,  2.84s/it]


Counter({0: 3675, 8: 3671, 1: 3667, 9: 3658, 5: 3655, 2: 3651, 4: 3648, 6: 3646, 7: 3644, 3: 3625})
Epoch 1/200
286/286 [==============================] - 11s 39ms/step - loss: 0.2760 - accuracy: 0.9191
Epoch 2/200
286/286 [==============================] - 11s 37ms/step - loss: 0.1481 - accuracy: 0.9485
Epoch 3/200
286/286 [==============================] - 11s 37ms/step - loss: 0.0794 - accuracy: 0.9709
Epoch 4/200
286/286 [==============================] - 11s 37ms/step - loss: 0.0669 - accuracy: 0.9754
Epoch 5/200
286/286 [==============================] - 11s 37ms/step - loss: 0.0592 - accuracy: 0.9782
Epoch 6/200
286/286 [==============================] - 11s 37ms/step - loss: 0.0558 - accuracy: 0.9811
Epoch 7/200
286/286 [==============================] - 10s 37ms/step - loss: 0.0539 - accuracy: 0.9805
Epoch 8/200
286/286 [==============================] - 11s 37ms/step - loss: 0.0479 - accuracy: 0.9827
Epoch 9/200
286/286 [==============================] - 11s 37ms/step - loss:

...

Counter({1: 3992, 9: 3988, 5: 3985, 0: 3976, 4: 3971, 6: 3968, 2: 3962, 7: 3947, 8: 3943, 3: 3941})
Epoch 1/200
310/310 [==============================] - 12s 40ms/step - loss: 0.0012 - accuracy: 0.9996
Epoch 2/200
310/310 [==============================] - 12s 37ms/step - loss: 8.3033e-04 - accuracy: 0.9998
Epoch 3/200
310/310 [==============================] - 12s 38ms/step - loss: 0.0015 - accuracy: 0.9995
Epoch 4/200
310/310 [==============================] - 12s 37ms/step - loss: 5.3153e-04 - accuracy: 0.9998
Epoch 5/200
310/310 [==============================] - 12s 37ms/step - loss: 5.3320e-04 - accuracy: 0.9998
Epoch 6/200
310/310 [==============================] - 11s 37ms/step - loss: 6.2673e-04 - accuracy: 0.9998
Epoch 7/200
310/310 [==============================] - 12s 38ms/step - loss: 5.4099e-04 - accuracy: 0.9999
Epoch 8/200
310/310 [==============================] - 12s 37ms/step - loss: 2.8366e-04 - accuracy: 0.9999
Epoch 9/200
310/310 [==============================]

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 13.33it/s]


Counter({1: 3992, 9: 3988, 5: 3987, 0: 3976, 4: 3972, 6: 3968, 2: 3964, 7: 3947, 8: 3943, 3: 3941})
Epoch 1/200
310/310 [==============================] - 12s 39ms/step - loss: 0.0011 - accuracy: 0.9996
Epoch 2/200
310/310 [==============================] - 12s 37ms/step - loss: 7.2099e-04 - accuracy: 0.9997
Epoch 3/200
310/310 [==============================] - 11s 37ms/step - loss: 7.6635e-04 - accuracy: 0.9997
Epoch 4/200
310/310 [==============================] - 12s 37ms/step - loss: 5.7660e-04 - accuracy: 0.9998
Epoch 5/200
310/310 [==============================] - 12s 37ms/step - loss: 4.8323e-04 - accuracy: 0.9998
Epoch 6/200
310/310 [==============================] - 12s 37ms/step - loss: 4.6393e-04 - accuracy: 0.9998
Epoch 7/200
310/310 [==============================] - 11s 37ms/step - loss: 2.4372e-04 - accuracy: 0.9999
Epoch 8/200
310/310 [==============================] - 12s 38ms/step - loss: 1.9099e-04 - accuracy: 0.9999
Epoch 9/200
310/310 [===========================

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 17.17it/s]


UnboundLocalError: local variable 'idxsample' referenced before assignment